In [9]:
import pandas as pd
from sqlalchemy import create_engine, text


def execute_query(conn, query):
    """Executes a SQL query using the provided connection and returns the results as a list of tuples."""
    result = conn.execute(text(query))
    columns = result.keys()
    rows = result.fetchall()
    return rows, columns


# Database connection details (you provided these)
server = '10.10.11.241'
database = 'omar.rme1'
user = 'omar'
password = 'omar123'

# Create SQLAlchemy engine
connection_string = f'mssql+pyodbc://{user}:{password}@{server}/{database}?driver=SQL+Server'
engine = create_engine(connection_string)

try:
    with engine.connect() as conn:
        print("Connected to SQL Server successfully!")

        # Get project_no input from the user
        project_no = input("Enter the project number: ")

        # Check if the project_no exists (handling both string and integer types)
        check_query = f"""
        SELECT 1
        FROM mat_mov
        WHERE CAST(project_no AS VARCHAR) = '{project_no}' 
        """
        check_result = conn.execute(text(check_query)).fetchone()

        if check_result:
            # Construct the SQL query with filters
            query = f"""
            SELECT *
            FROM mat_mov
            WHERE trx_type IN ('Move Order Issue on Project', 'RME Issue ( On Project)', 'RME Site Return')
            AND CAST(project_no AS VARCHAR) = '{project_no}'
            """

            # Execute the query and fetch data
            results, columns = execute_query(conn, query)

            # Create a Pandas DataFrame
            df = pd.DataFrame(results)

            # Check if the DataFrame is empty
            if df.empty:
                print(
                    f"No data found for project number: {project_no} with the specified trx_type values."
                )
            else:
                df.columns = columns
                # Dynamically create the Excel file name
                excel_file_name = f"mat_mov_{project_no}.xlsx"
                df.to_excel(excel_file_name, index=False)
                print(f"Data exported to {excel_file_name} successfully!")
        else:
            print(
                f"Project number '{project_no}' does not exist in the database."
            )

except Exception as e:
    print("Error:", e)

Connected to SQL Server successfully!
Data exported to mat_mov_157.xlsx successfully!
